In [1]:
import numpy as np
from utils import *
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from scipy.misc import logsumexp
# np.seterr(all='raise')

In [2]:
# superclass of modules
class Module:
    """
    Module is a super class. It could be a single layer, or a multilayer perceptron.
    """
    
    def __init__(self):
        self.train = True
        return
    
    def forward(self, _input):
        """
        h = f(z); z is the input, and h is the output.
        
        Inputs:
        _input: z
        
        Returns:
        output h
        """
        pass
    
    def backward(self, _input, _gradOutput):
        """
        Compute:
        gradient w.r.t. _input
        gradient w.r.t. trainable parameters
        
        Inputs:
        _input: z
        _gradOutput: dL/dh
        
        Returns:
        gradInput: dL/dz
        """
        pass
        
    def parameters(self):
        """
        Return the value of trainable parameters and its corresponding gradient (Used for grandient descent)
        
        Returns:
        params, gradParams
        """
        pass
    
    def training(self):
        """
        Turn the module into training mode.(Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = True
        
    def evaluate(self):
        """
        Turn the module into evaluate mode.(Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = False
        

In [3]:
class Sequential(Module):
    """
    Sequential provides a way to plug layers together in a feed-forward manner.
    """
    def __init__(self):
        Module.__init__(self)
        self.layers = [] # layers contain all the layers in order
    
    def add(self, layer):
        self.layers.append(layer) # Add another layer at the end
    
    def size(self):
        return len(self.layers) # How many layers.
    
    def forward(self, _input):
        """
        Feed forward through all the layers, and return the output of the last layer
        """
        # self._inputs saves the input of each layer
        # self._inputs[i] is the input of i-th layer
        self._inputs = [_input]
        
        # YOUR CODE HERE
        for i in range(self.size()):
            current_input = self._inputs[i]
            layer = self.layers[i]
            self._inputs.append(layer.forward(current_input))
        
        # The last element of self._inputs is the output of last layer
        self._output = self._inputs[-1]
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        Backpropogate through all the layers using chain rule.
        """
        # self._gradInputs[i] is the gradient of loss w.r.t. the input of i-th layer
        self._gradInputs = [None] * (self.size() + 1)
        self._gradInputs[self.size()] = _gradOutput

        # YOUR CODE HERE
        for i in range(self.size(), 0, -1):
#             print("{} layer".format(i))
            current_input = self._inputs[i - 1]
#             print("length of gradinput {}".format(len(self._gradInputs)))
            current_gradoutput = self._gradInputs[i]
            layer = self.layers[i - 1]
            gradInput = layer.backward(current_input, current_gradoutput)
            self._gradInputs[i - 1] = gradInput
        
        self._gradInput = self._gradInputs[0]
        return self._gradInput
    
    def parameters(self):
        """
        Return trainable parameters and its corresponding gradient in a nested list
        """
        params = []
        gradParams = []
        for m in self.layers:
            _p, _g = m.parameters()
            if _p is not None:
                params.append(_p)
                gradParams.append(_g)
        return params, gradParams

    def training(self):
        """
        Turn all the layers into training mode
        """
        Module.training(self)
        for m in self.layers:
            m.training()
    
    def evaluate(self):
        """
        Turn all the layers into evaluate mode
        """
        Module.evaluate(self)
        for m in self.layers:
            m.evaluate()
        

In [4]:
class FullyConnected(Module):
    """
    Fully connected layer
    """
    def __init__(self, inputSize, outputSize):
        Module.__init__(self)
        # Initalization
        stdv = 1./np.sqrt(inputSize)
        
#         self.weight = np.random.uniform(-stdv, stdv, (inputSize, outputSize))
#         self.gradWeight = np.ndarray((inputSize, outputSize))
        
        self.weight = np.random.uniform(-stdv, stdv, (outputSize, inputSize))
#         self.weight = np.random.uniform(0, 1, (outputSize, inputSize))
#         self.weight = (1.0/2) * np.ones([outputSize, inputSize])
        self.gradWeight = np.ndarray((outputSize, inputSize))
        self.bias = np.random.uniform(-stdv, stdv, outputSize)
#         self.bias = (1.0/2) * np.ones(outputSize)
        self.gradBias = np.ndarray(outputSize)
        
    def forward(self, _input):
        """
        output = W * input + b
        
        _input:
        N x inputSize matrix
        
        """
        # YOUR CODE HERE
        N = _input.shape[0]
#         outputSize = self.bias.shape[1]
        self._output = np.dot(self.weight, _input.T).T + self.bias
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        _input:
        N x inputSize matrix
        _gradOutputSize:
        N x outputSize matrix
        """
        # YOUR CODE HERE
        N = _input.shape[0]
        inputSize = _input.shape[1]
        outputSize = _gradOutput.shape[1]
        gradWeight_ = [np.dot(_gradOutput[i].reshape([outputSize, 1]), _input[i].reshape([1, inputSize]))
                          for i in range(N)]
        # sum or mean?
        self.gradWeight = np.sum(gradWeight_, axis = 0)
        self.gradBias = np.sum(_gradOutput, axis = 0)
        
        self._gradInput = np.dot(self.weight.T, _gradOutput.T).T
        return self._gradInput
        
    def parameters(self):
        """
        Return weight and bias and their g
        """
        return [self.weight, self.bias], [self.gradWeight, self.gradBias]

In [5]:
class ReLU(Module):
    """
    ReLU activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        """
        output = max(0, input)
        
        _input:
        N x d matrix
        """
        #YOUR CODE HERE
        self._output = np.maximum(_input, np.zeros_like(_input))
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        gradInput = gradOutput * mask
        mask = _input > 0
        
        _input:
        N x d matrix
        
        _gradOutput:
        N x d matrix
        """
        # YOUR CODE HERE
        mask = (_input > 0).astype(int)
        self._gradInput = np.multiply(_gradOutput, mask)
        return self._gradInput
        
    def parameters(self):
        """
        No trainable parametersm, return None
        """
        return None, None

In [6]:
# Optional
class Logistic(Module):
    """
    Logistic activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        self._output = 0 #YOUR CODE HERE
        return self._output
    
    def backward(self, _input, _gradOutput):
        self._gradInput = 0 #YOUR CODE HERE
        return self._gradInput
        
    def parameters(self):
        return None, None

In [7]:
# Optional
class Dropout(Module):
    """
    A dropout layer
    """
    def __init__(self, p = 0.5):
        Module.__init__(self)
        self.p = p #self.p is the drop rate, if self.p is 0, then it's a identity layer
        
    def forward(self, _input):
        self._output = _input
        # YOUR CODE HERE
        # Need to take care of training mode and evaluation mode
        return self._output
    
    def backward(self, _input, _gradOutput):
        self._gradInput = _gradOutput
        #YOUR CODE HERE
        return self._gradInput
    
    def parameters(self):
        """
        No trainable parameters.
        """
        return None, None

In [8]:
class SoftMaxLoss(object):
    def __init__(self):
        return
        
    def forward(self, _input, _label):
        """
        Softmax and cross entropy loss layer. Should return a scalar, since it's a
        loss. (It's almost identical to what in hw2)

        _input: N x C
        _labels: N x C, one-hot

        Returns: loss (scalar)
        """
        #YOUR CODE HERE
        N = _input.shape[0]
        C = _input.shape[1]
#         expinput = np.exp(_input - np.repeat(_input.max(axis = 1).reshape([N, 1]), C, axis = 1))
#         sum_ = np.repeat(np.sum(expinput, axis = 1).reshape(N, 1), C, axis = 1)
#         prob = np.divide(expinput, sum_)
#         loss = np.log(prob)
        self._output = np.sum(-np.multiply(_input, _label).sum(axis=1) + logsumexp(_input, axis=1))
#         self._output = -np.sum(np.multiply(loss, _label))
        return self._output
    
    def backward(self, _input, _label):
        #YOUR CODE HERE
        N = _input.shape[0]
        C = _input.shape[1]
        expinput = np.exp(_input - np.repeat(_input.max(axis = 1).reshape([N, 1]), C, axis = 1))
        sum_ = np.repeat(np.sum(expinput, axis = 1).reshape(N, 1), C, axis = 1)
        prob = np.divide(expinput, sum_)
#         grad_ = np.sum(np.multiply(_label, prob))
#         self._gradInput = np.sum(_label, axis = 0) + grad_
#         self._gradInput = np.sum(-_label + prob, axis = 0)
        self._gradInput = -_label + prob        
        return self._gradInput

In [9]:
# Test softmaxloss, the relative error should be small enough
def test_sm():
    crit = SoftMaxLoss()
    gt = np.zeros((3, 5))
    gt[np.arange(3), np.array([1,2,3])] = 1

    x = np.random.random((3,5))

    def test_f(x):
        return crit.forward(x, gt)

    gradInput = crit.backward(x, gt)

    gradInput_num = numeric_gradient(test_f, x, 1, 1e-6)
#     gradInput_num = np.sum(gradInput_num, axis = 0)
#     print(gradInput)
#     print(gradInput_num)
    print(relative_error(gradInput, gradInput_num, 1e-8))
    
test_sm()


1.02400457377e-09


In [10]:
# Test modules, all the relative errors should be small enough
def test_module(model):

    model.evaluate()

    crit = TestCriterion()
    gt = np.random.random((3, 10))
    x = np.random.random((3, 10))
    def test_f(x):
        return crit.forward(model.forward(x), gt)
#     print("gt {}".format(gt))
#     print("x {}".format(x))
    gradInput = model.backward(x, crit.backward(model.forward(x), gt))
    gradInput_num = numeric_gradient(test_f, x, 1, 1e-6)
    print(relative_error(gradInput, gradInput_num, 1e-8))

# Test fully connected
print("FC")
model = FullyConnected(10, 10)
test_module(model)

# Test ReLU
print("Relu")
model = ReLU()
test_module(model)

# Test Dropout
# model = Dropout()
# test_module(model)
# You can only test dropout in evaluation mode.

# Test Sequential
print("Sequential")
model = Sequential()
model.add(FullyConnected(10, 10))
model.add(ReLU())
# model.add(FullyConnected(10, 1))
#model.add(Dropout())
test_module(model)

FC
2.34181949214e-08
Relu
7.36022825478e-10
Sequential
5.47673806655e-09


In [11]:
# Test gradient descent, the loss should be lower and lower
trainX = np.random.random((10,5))
# trainX = np.ones([10, 3])

model = Sequential()
model.add(FullyConnected(5, 3))
model.add(ReLU())
#model.add(Dropout())
model.add(FullyConnected(3, 1))

crit = TestCriterion()

params, gradParams = model.parameters()
print(params)

it = 0
state = None
while True:
    output = model.forward(trainX)
#     print(output)
    loss = crit.forward(output, None)
    doutput = crit.backward(output, None)
    if it % 100 == 0:
        print("loss = {}".format(loss))
#         print("gradoutput = {}".format(doutput))
    model.backward(trainX, doutput)
    params, gradParams = model.parameters()
#     print("gradparams: {}".format(gradParams))
#     sgd(params, gradParams, 0.01, 0.8, state)
    sgd(params, gradParams, 0.01, 0.8)
    if it > 1000:
        break
    it += 1
    

[[array([[ 0.308034  ,  0.11320524, -0.33503905,  0.12735072,  0.18311318],
       [-0.28412608,  0.1025937 ,  0.35183396, -0.18996367,  0.05782674],
       [-0.35328554, -0.29711092,  0.31444612, -0.30638926, -0.17612341]]), array([-0.17429204,  0.3716292 , -0.04318029])], [array([[ 0.38262956, -0.12564589,  0.44585997]]), array([-0.5700844])]]
loss = 0.586448103438
loss = 0.0525713179881
loss = 0.0167324519869
loss = 0.00744902346675
loss = 0.00413410494975
loss = 0.00407916581178
loss = 0.00424292761433
loss = 0.00368156660255
loss = 0.00477680385828
loss = 0.00361584224503
loss = 0.00467094292296


Now we start to work on real data.

In [12]:
import MNIST_utils
data_fn = "CLEAN_MNIST_SUBSETS.h5"

# We only consider large set this time
print("Load large trainset.")
Xlarge,Ylarge = MNIST_utils.load_data(data_fn, "large_train")
print(Xlarge.shape)
print(Ylarge.shape)

print("Load valset.")
Xval,Yval = MNIST_utils.load_data(data_fn, "val")
print(Xval.shape)
print(Yval.shape)

Load large trainset.
(8000, 576)
(8000, 10)
Load valset.
(2000, 576)
(2000, 10)


In [13]:
def predict(X, model):
    """
    Evaluate the soft predictions of the model.
    Input:
    X : N x d array (no unit terms)
    model : a multi-layer perceptron
    Output:
    yhat : N x C array
        yhat[n][:] contains the score over C classes for X[n][:]
    """
    return model.forward(X)

def error_rate(X, Y, model):
    """
    Compute error rate (between 0 and 1) for the model
    """
    model.evaluate()
    res = 1 - (model.forward(X).argmax(-1) == Y.argmax(-1)).mean()
    model.training()
    return res

from copy import deepcopy

def runTrainVal(X,Y,model,Xval,Yval,trainopt):
    """
    Run the train + evaluation on a given train/val partition
    trainopt: various (hyper)parameters of the training procedure
    During training, choose the model with the lowest validation error. (early stopping)
    """
    
    params, gradParams = model.parameters()
    
    eta = trainopt['eta']
    
    N = X.shape[0] # number of data points in X
    
    # Save the model with lowest validation error
    minValError = np.inf
    saved_model = None
    
    shuffled_idx = np.random.permutation(N)
    start_idx = 0
    for iteration in range(trainopt['maxiter']):
        if iteration % int(trainopt['eta_frac'] * trainopt['maxiter']) == 0:
            eta *= trainopt['etadrop']
        # form the next mini-batch
        stop_idx = min(start_idx + trainopt['batch_size'], N)
        batch_idx = range(N)[int(start_idx):int(stop_idx)]
        bX = X[shuffled_idx[batch_idx],:]
        bY = Y[shuffled_idx[batch_idx],:]

        score = model.forward(bX)
        loss = crit.forward(score, bY)
        # print(loss)
        dscore = crit.backward(score, bY)
        
#         print("bX shape {} and dscore shape {}".format(bX.shape, dscore.shape))
        model.backward(bX, dscore)
        
        # Update the data using 
        sgd(params, gradParams, eta, weight_decay = trainopt['lambda'])    
        start_idx = stop_idx % N
        
        if (iteration % trainopt['display_iter']) == 0:
            #compute train and val error; multiply by 100 for readability (make it percentage points)
            trainError = 100 * error_rate(X, Y, model)
            valError = 100 * error_rate(Xval, Yval, model)
            print('{:8} batch loss: {:.3f} train error: {:.3f} val error: {:.3f}'.format(iteration, loss, trainError, valError))
            
            if valError < minValError:
                saved_model = deepcopy(model)
                minValError = valError
        
    return saved_model, minValError, trainError

In [14]:
def build_model(input_size, hidden_size, output_size, activation_func = 'ReLU', dropout = 0):
    """
    Build the model:
    input_size: the dimension of input data
    hidden_size: the dimension of hidden vector
    output_size: the output size of final layer.
    activation_func: ReLU, Logistic, Tanh, etc. (Need to be implemented by yourself)
    dropout: the dropout rate: if dropout == 0, this is equivalent to no dropout
    """
    #YOUR CODE HERE
    model = Sequential()
    model.add(FullyConnected(input_size, hidden_size))
    model.add(ReLU())
#     for i in range(0):
#         model.add(FullyConnected(hidden_size, hidden_size))
#         if activation_func == 'ReLU':
#             model.add(ReLU())
    model.add(FullyConnected(hidden_size, output_size))
    
    return model
    

In [15]:
# -- training options
trainopt = {
    'eta': .1,   # initial learning rate
    'maxiter': 20000,   # max number of iterations (updates) of SGD
    'display_iter': 500,  # display batch loss every display_iter updates
    'batch_size': 100,  
    'etadrop': .5, # when dropping eta, multiply it by this number (e.g., .5 means halve it)
    'eta_frac': .25  #
}

NFEATURES = Xlarge.shape[1]

# we will maintain a record of models trained for different values of lambda
# these will be indexed directly by lambda value itself
trained_models = dict()

# set the (initial?) set of lambda values to explore
lambdas = np.array([0, 0.001, 0.01, 0.1])
hidden_sizes = np.array([10])
    
for lambda_ in lambdas:
    for hidden_size_ in hidden_sizes:
        trainopt['lambda'] = lambda_
        model = build_model(NFEATURES, hidden_size_, 10, dropout = 0)
        crit = SoftMaxLoss()
        # -- model trained on large train set
        trained_model,valErr,trainErr = runTrainVal(Xlarge, Ylarge, model, Xval, Yval, trainopt)
        trained_models[(lambda_, hidden_size_)] = {'model': trained_model, "val_err": valErr, "train_err": trainErr }
        print('train set model: -> lambda= %.4f, train error: %.2f, val error: %.2f' % (lambda_, trainErr, valErr))
    
best_trained_lambda = 0.
best_trained_model = None
best_trained_val_err = 100.
for (lambda_, hidden_size_), results in trained_models.items():
    print('lambda= %.4f, hidden size: %5d, val error: %.2f' %(lambda_, hidden_size_, results['val_err']))
    if results['val_err'] < best_trained_val_err:
        best_trained_val_err = results['val_err']
        best_trained_model = results['model']
        best_trained_lambda = lambda_

print("Best train model val err:", best_trained_val_err)
print("Best train model lambda:", best_trained_lambda)

       0 batch loss: 233.299 train error: 91.000 val error: 92.000
     500 batch loss: 230.757 train error: 91.000 val error: 92.000
    1000 batch loss: 229.980 train error: 91.000 val error: 92.000
    1500 batch loss: 229.673 train error: 91.000 val error: 92.000
    2000 batch loss: 231.741 train error: 91.000 val error: 92.000


KeyboardInterrupt: 

In [ ]:
#Generate a Kaggle submission file using `model`
kaggleX = MNIST_utils.load_data(data_fn, 'kaggle')
kaggleYhat = predict(kaggleX, best_trained_model).argmax(-1)
save_submission('submission-mnist.csv', kaggleYhat)